In [ ]:
import duckdb

### Add Kentucky Boundary

Add Kentucky Boundary from _kybnd.parquet_ to latest.ddb main tables

I will be using `with duckdb.connect()` in order to have connections close when queries are complete.  
This will allow me to test **latest.ddb** in DBeaver UI.

In [ ]:
with duckdb.connect('../latest.ddb') as conn:
    conn.sql("""
             INSTALL spatial;
             LOAD spatial;

             CREATE OR REPLACE TABLE ky_bnd AS
             SELECT * 
             FROM read_parquet('../parquets/kybnd.parquet')
             """)
    
    print('Table ky_bnd created.')

### Kentucky Extent

calculated min/max from state boundary ky_bnd table.
`-89.57122, 36.49706, -81.96479, 39.14774`

bbox sql:

```sql
    bbox.xmin BETWEEN -89.57122 AND -81.96479
    AND bbox.ymin BETWEEN 36.49706 AND 39.14774
```


In [ ]:
# Tables where Kentucky can be queried from the table
# eliminating the need to run a spatial query on Ky's 
# border geometry.
# bbox will be used to narrow down the field of view.


with duckdb.connect('../latest.ddb') as conn:
  conn.sql("""
  LOAD spatial;
  
  CREATE OR REPLACE TABLE place as
  WITH kybbox as (
        SELECT * FROM place_vw
        WHERE bbox.xmin BETWEEN -89.57122 AND -81.96479
          AND bbox.ymin BETWEEN 36.49706 AND 39.14774
        )
        SELECT * from kybbox,  
          UNNEST(kybbox.addresses) as a                    
        WHERE a.unnest.region = 'KY'
  """)

  print('Table place created.\n')


### Clip to Kentucky Border

These tables will need further processsing.  

 - Step 1: query to bounding box
 - Step2:  clip to Kentucky boundary

In [ ]:
with duckdb.connect('../latest.ddb') as conn:
    conn.sql("""
        LOAD spatial;
        CREATE OR REPLACE TABLE infrastructure AS
        SELECT * 
        FROM infrastructure_vw
        WHERE bbox.xmin BETWEEN -89.57122 AND -81.96479
            AND bbox.ymin BETWEEN 36.49706 AND 39.14774;
""")

        # COPY (
        #      SELECT * from infrastructure i, ky_bnd k
        #      WHERE ST_Intersects(i.geometry, k.geometry)
        #      ) TO '../parquets/infrastructure.parquet'
        #      WITH (FORMAT 'parquet');